In [10]:
import json
from pprint import pprint
import tinydb as tdb


## OSM: Requete OverPass turbo:

```sql
[out:json][timeout:500];
{{geocodeArea:France}}->.boundaryarea;

(
 node(area.boundaryarea)["ref:mhs"];
 way(area.boundaryarea)["ref:mhs"];
 relation(area.boundaryarea)["ref:mhs"];
);

out tags;
```
-- 20Mo

Rq: j'ai essayé avec `out meta;`, le fichier fait ~200Mo, et avec `out body;` ~90Mo... c'est surement mieux de séparer par département...


In [11]:
# Charge le fichier JSON:
filename = 'data/osmTags_France.json' 

with open(filename) as f:
    dataOsm = json.load(f)
    
dataOsm = dataOsm['elements']
len(dataOsm)

33071

In [12]:
# Garde seulement les monuments avec le code commençant par PA...
dataOsm = [ el for el in dataOsm if el['tags']['ref:mhs'].startswith('PA') ]
len(dataOsm)

32246

In [6]:
dataOsm[0]

{'id': 21609292,
 'tags': {'heritage': '3',
  'heritage:operator': 'mhs',
  'historic': 'memorial',
  'image': "http://phototheque.1000wallpapers.com/galleries/Nature/p/Pa+%E0+Pl/Parc+de+la+T%EAte+d'Or/Ile+du+souvenir/Monument+aux+morts/large/Monument+aux+morts+5.jpg",
  'mhs:inscription_date': '1982',
  'name': "Monument aux morts de l'île du Souvenir",
  'ref:mhs': 'PA00117982',
  'wikidata': 'Q17341711',
  'wikipedia': "fr:Monument aux morts de l'île du Souvenir"},
 'type': 'node'}

In [7]:
def flatten( dico ):
    
    flatdico = dico['tags']
    flatdico['idOSM'] = dico['id']
    flatdico['typeOSM'] = dico['type']
    
    return flatdico

In [8]:
# Garde que les tags - and flatten the dict 
dataOsm = [ flatten( el ) for el in dataOsm ]

In [9]:
dataOsm[:2]

[{'heritage': '3',
  'heritage:operator': 'mhs',
  'historic': 'memorial',
  'idOSM': 21609292,
  'image': "http://phototheque.1000wallpapers.com/galleries/Nature/p/Pa+%E0+Pl/Parc+de+la+T%EAte+d'Or/Ile+du+souvenir/Monument+aux+morts/large/Monument+aux+morts+5.jpg",
  'mhs:inscription_date': '1982',
  'name': "Monument aux morts de l'île du Souvenir",
  'ref:mhs': 'PA00117982',
  'typeOSM': 'node',
  'wikidata': 'Q17341711',
  'wikipedia': "fr:Monument aux morts de l'île du Souvenir"},
 {'idOSM': 26691547,
  'name': 'Bailleul',
  'name:fr': 'Bailleul',
  'name:nl': 'Belle',
  'name:vls': 'Belle',
  'place': 'town',
  'population': '14480',
  'ref:FR:SIREN': '215900432',
  'ref:INSEE': '59043',
  'ref:mhs': 'PA00107358',
  'source:population': 'INSEE 2013',
  'typeOSM': 'node',
  'wikidata': 'Q2481155'}]

In [94]:
# Push to tinyDB
db = tdb.TinyDB('data/tinyDB.json')
db.purge_table('osm')
osm = db.table('osm')
ids = osm.insert_multiple(dataOsm)
len(ids)

32246

## Requete Wikidata
```sql
SELECT DISTINCT ?item ?itemDescription ?merimee ?article  WHERE {
  ?item wdt:P380 ?merimee .
  
  OPTIONAL { ?article schema:about ?item .
  ?article schema:isPartOf <https://fr.wikipedia.org/> . }
   
  FILTER(STRSTARTS(?merimee, "PA")).
  
  SERVICE wikibase:label {
     bd:serviceParam wikibase:language "fr" .
 }
}
```
-- 7,6 Mo

In [16]:
# Charge le fichier JSON:
filename = 'data/wikidata_France.json' 

with open(filename) as f:
    dataWiki = json.load(f)
    
len(dataWiki)

44912

In [17]:
dataWiki[0]

{'article': 'https://fr.wikipedia.org/wiki/%C3%89glise_Saint-%C3%89tienne-du-Mont_de_Paris',
 'item': 'http://www.wikidata.org/entity/Q583251',
 'itemDescription': 'édifice religieux de Paris, en France',
 'merimee': 'PA00088414'}

In [18]:
# Push to tinyDB
db = tdb.TinyDB('data/tinyDB.json')
db.purge_table('wikidata')
wikidata = db.table('wikidata')
ids = wikidata.insert_multiple(dataWiki)
len(ids)

44912

In [21]:
print( len(dataWiki) )
print( len(dataWiki) - len(dataOsm) )

44912
12666


Il semble bien avoir tout les monuments de Mérimés dans Wikidata...  
et il manquerait 13552 monuments dans OSM ?

## Monuments sans page Wikipedia et absent de OSM

In [22]:
noWikiAndNoOsm = [ d for d in withoutAwikiPage if d['merimee'] not in dataOsm ]
len(noWikiAndNoOsm)

23206

In [23]:
noWikiAndNoOsm[0]

{'item': 'http://www.wikidata.org/entity/Q1440262',
 'itemDescription': 'église à Vaugines (Vaucluse)',
 'merimee': 'PA84000019'}

## Monuments AVEC une page Wikipedia mais absent de OSM

In [24]:
wikiButNoOsm = [ d for d in dataWiki if 
                    (d['merimee'] not in dataOsm) and ('article' in d) ]
len(wikiButNoOsm)

21706

In [25]:
wikiButNoOsm[0]

{'article': 'https://fr.wikipedia.org/wiki/%C3%89glise_Saint-%C3%89tienne-du-Mont_de_Paris',
 'item': 'http://www.wikidata.org/entity/Q583251',
 'itemDescription': 'édifice religieux de Paris, en France',
 'merimee': 'PA00088414'}

# Monument en double dans Wikidata

In [53]:
wikiRefs = {}
for i, d in enumerate(dataWiki):
    merimee = d['merimee']
    if merimee in wikiRefs:
        wikiRefs[merimee].append(i)
    else:
        wikiRefs[merimee] = [i ]
        
listOfDuplicates = [ (refs, indices) for refs, indices in wikiRefs.items() if len(indices)>1 ]

listOfDuplicates[:2]

[('PA00088789', [696, 8528]), ('PA00102942', [6330, 9874])]

In [54]:
def getWikiCode(item):
    item = item.split('/')
    return item[-1]

def formatWikiInfo( info ):
    code = getWikiCode( info['item'] )
    desc = info['itemDescription']
    return '  %s - %s' % (code, desc)

for merimee, indices in listOfDuplicates:
    print(merimee + ':')
    print( '\n'.join( [formatWikiInfo(dataWiki[i]) for i in indices]))

PA00088789:
  Q769842 - chapelle située à Paris, en France
  Q3279847 - maison dans le 7e arrondissement de Paris
PA00102942:
  Q1518963 - tour à Aigues-Mortes (Gard)
  Q3424607 - rempart à Aigues-Mortes (Gard)
PA00114885:
  Q2971879 - château situé dans l'Oise, en France
  Q3424631 - rempart de ville à Senlis (Oise)
PA00088431:
  Q3145813 - hôtel particulier dans le 5e arrondissement de Paris
  Q2669368 - thermes à Paris
PA00116455:
  Q3145289 - hôpital à Pérouges (Ain)
  Q3424623 - enceinte de Pérouges, France
PA64000047:
  Q12252858 - pont à Bidache (Pyrénées-Atlantiques)
  Q27890674 - moulin à Bidache (Pyrénées-Atlantiques)
PA00091138:
  Q11682090 - menhir de Carnac, France
  Q15930186 - site mégalithique de Carnac, France
  Q24931935 - mégalithe à Carnac (Morbihan)
PA00112275:
  Q15950349 - hôpital à Dijon (Côte-d'Or)
  Q17177229 - chapelle à Dijon (Côte-d'Or)
PA00091075:
  Q1739261 - alignements mégalithiques de Carnac, France
  Q15791807 - allée couverte de Carnac, France
PA0008